In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
import scipy

import sys
sys.path.append('../../..')
import mutagenesisfunctions as mf
import bpdev as bd
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

from Bio import AlignIO
import time as time
import pandas as pd
np.random.seed(42)

In [23]:
def vis_contacts(fam):

    ######### EXTRACT DATA ###########

    #extract SoM results after an APC correction
    arrayspath = 'Arrays/%s_mlp_%s.npy'%(fam,numhidden)
    hol_mut2 = np.load(arrayspath)
    seqlen,_, dims,_ = hol_mut2.shape
    C = bd.get_wc(arrayspath, seqlen, dims, bpugSQ=0, denoise='APC')

    #Load in the EC annotation
    ECannotfile = '%s/%s.EC.interaction.txt'%(fam,fam) 
    EC_df = pd.read_csv(ECannotfile, delimiter='\s+')

    ######### QUALITATIVE CONTACTS ###########

    #Get the top L/2 SoM scores
    bp_shade = bd.plot_wcrank(C, seqlen, seqlen//2)
    bp_shade[np.tril_indices(seqlen)] = 0.
    SoM_top = np.where(bp_shade > 0.)
    #Top L/2 EC scores
    topEC = EC_df.loc[:seqlen//2, ['Rfam_reduced_position1', 'Rfam_reduced_position2']]
    #All cWW annotated interactions
    cWW_int = EC_df[EC_df['interactions'] == 'cWW']
    #All PDB contacts < 8 angstroms
    close_cont = EC_df[EC_df['minimum_atom_distance'] <= 8.0]

    #plot
    fig = plt.figure(figsize=(24,7))
    ax1 = fig.add_subplot(1,3,1)
    ax1.scatter(SoM_top[0], SoM_top[1], c='y', label = 'SoM')
    ax1.scatter(topEC.iloc[:,1], topEC.iloc[:,0], c='r', label = 'EC')
    ax1.set_title('SoM vs. top L/2 ECs')
    ax1.legend()

    ax2 = fig.add_subplot(1,3,2)
    ax2.scatter(SoM_top[0], SoM_top[1], c='y', label = 'SoM')
    ax2.scatter(cWW_int.iloc[:,2], cWW_int.iloc[:,1], c='b', label = 'cWW')
    ax2.set_title('SoM vs. all cWW contacts')
    ax2.legend()

    ax = fig.add_subplot(1,3,3)
    ax.scatter(SoM_top[0], SoM_top[1], c='y', label = 'SoM')
    ax.scatter(close_cont.iloc[:,2], close_cont.iloc[:,1], c='g', label = 'PDB contacts')
    ax.set_title('SoM vs. contacts < 8A')
    ax.legend()
    plt.axis('equal')
    plt.savefig('%s/Images_mlp/%s_contacts_vis.png'%(fam,fam))
    plt.savefig('Contacts_vis/%s_contacts_vis.png'%(fam))

    ######### QUANTITATIVE GRAPHS ###########

    #Get the real positive and negative pdb contacts
    real_pos = np.zeros((seqlen,seqlen))
    for ii in range(len(close_cont)):
        real_pos[close_cont.iloc[ii, 1], close_cont.iloc[ii, 2]] = 1.
    real_neg = np.ones((seqlen,seqlen))
    for ii in range(len(close_cont)):
        real_neg[close_cont.iloc[ii, 1], close_cont.iloc[ii, 2]] = 0.

    def sumstats(thresh):
        #Get the top thresh SoM scores
        som_pos = bd.plot_wcrank(C, seqlen, thresh*2)
        som_pos[np.tril_indices(seqlen)] = 0.
        #Get the bottom thresh SoM scores
        som_neg = (~som_pos.astype('bool'))*1 #incantation converts binary to boolean, flips them then back to binary

        TP = np.sum(real_pos*som_pos)
        FP = np.sum(real_neg*som_pos)
        TN = np.sum(real_neg*som_neg)
        FN = np.sum(real_pos*som_neg)

        PPV = TP/(TP+FP)
        FDR = FP/(TP+FP)
        TPR = TP/(TP+FN)

        return (PPV, FDR, TPR, TP, FP, TN, FN)

    def sumstats_EC(thresh):
        #Top thresh EC scores
        n_EC_P = len(EC_df.iloc[:thresh, :])
        EC_pos = np.zeros((seqlen,seqlen))
        for ii in range(n_EC_P):
            EC_pos[EC_df.iloc[ii, 1], EC_df.iloc[ii, 2]] = 1.
        EC_neg = np.ones((seqlen,seqlen))
        for ii in range(n_EC_P):
            EC_neg[EC_df.iloc[ii, 1], EC_df.iloc[ii, 2]] = 0.

        TP = np.sum(real_pos*EC_pos)
        FP = np.sum(real_neg*EC_pos)
        TN = np.sum(real_neg*EC_neg)
        FN = np.sum(real_pos*EC_neg)

        PPV = TP/(TP+FP)
        FDR = FP/(TP+FP)
        TPR = TP/(TP+FN)

        return (PPV, FDR, TPR, TP, FP, TN, FN)

    extent = seqlen

    EC_ppv = []
    EC_tpr = []
    SoM_ppv = []
    SoM_tpr = []
    for tr in range(extent):
        ecst = sumstats_EC(tr)
        somst = sumstats(tr)

        EC_ppv.append(ecst[0])
        EC_tpr.append(ecst[2])

        SoM_ppv.append(somst[0])
        SoM_tpr.append(somst[2])

    fig = plt.figure(figsize=(12,5))
    ax1 = fig.add_subplot(1,2,1)
    ax1.plot(range(extent),SoM_ppv, label='SoM')
    ax1.plot(range(extent),EC_ppv, label='EC')
    ax1.set_xlabel('Number of contact predicted')
    ax1.set_ylabel('Positive Predictive Value')
    ax1.legend()

    ax2 = fig.add_subplot(1,2,2)
    ax2.plot(range(extent),SoM_tpr, label='SoM')
    ax2.plot(range(extent),EC_tpr, label='EC')
    ax2.set_xlabel('Number of contact predicted')
    ax2.set_ylabel('True Positive Rate')
    ax2.legend()
    plt.savefig('%s/Images_mlp/%s_stats.png'%(fam,fam))
    plt.savefig('Stats_graphs/%s_stats.png'%(fam))
    
    ######### QUANTITATIVE CONTACTS ###########

In [24]:
#Load the SoM results file
families = ['RF00002', 'RF00005', 'RF00010', 'RF00017', 'RF00023', 'RF00050',
            'RF00059', 'RF00162', 'RF00167', 'RF00169', 'RF00174', 'RF00234',
            'RF00380', 'RF00504', 'RF01734', 'RF01786', 'RF01831', 'RF01852', 'RF01960', 'RF02001']

numhidden = 512

In [26]:
for fam in families:
    vis_contacts(fam)
    print (fam)

/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:94: RuntimeWarning: invalid value encountered in double_scalars
/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:95: RuntimeWarning: invalid value encountered in double_scalars
/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in double_scalars
/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:74: RuntimeWarning: invalid value encountered in double_scalars


RF00002
RF00005
RF00010
RF00017
RF00023
RF00050


/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


RF00059
RF00162
RF00167
RF00169
RF00174
RF00234
RF00380
RF00504
RF01734
RF01786
RF01831
RF01852


IOError: [Errno 2] No such file or directory: 'Arrays/RF01960_mlp_512.npy'

In [28]:
vis_contacts('RF02001')
print ('RF02001')

/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:94: RuntimeWarning: invalid value encountered in double_scalars
/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:95: RuntimeWarning: invalid value encountered in double_scalars
/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in double_scalars
/home/peter/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:74: RuntimeWarning: invalid value encountered in double_scalars


RF02001
